## Text Summrization
- 文章簡化，有分成2成類型，extractiom、abstraction，本篇是用extraction，可以透過預訓練模型去達成。 文章簡化其實本質的意思是說，哪些句子與文章本身相似度高，可以具有代表性，依據這個思路就可以直覺地完成這個任務。
- 效益、解決的問題
    - 節省閱讀時間，讀取重點。
- 步驟: 
    1. 將文章向量化得到doc2vec
        - 若文章太長可以分段處理，透過向量在**平均化pooling**。
    2. 將文章拆解成句子(中文透過，。)
    3. 將句子向量化得到sentence2vec
    4. 計算文章與句子之間的相似度(cosine similarity or others)
    5. 根據使用需求，取得句子(如 top n 個句子 / 或者相似度超過 X % 的所有句子)

In [15]:
import transformers
from transformers import BertTokenizer, TFBertModel
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
from sklearn.metrics.pairwise import cosine_similarity
%matplotlib inline

In [16]:
# 機器學習的wiki定義
doc = """機器學習是人工智慧的一個分支。人工智慧的研究歷史有著一條從以「推理」為重點，到以「知識」為重點，再到以「學習」為重點的自然、清晰的脈絡。顯然，機器學習是實現人工智慧的一個途徑，即以機器學習為手段解決人工智慧中的問題。機器學習在近30多年已發展為一門多領域交叉學科，涉及機率論、統計學、逼近論、凸分析、計算複雜性理論等多門學科。機器學習理論主要是設計和分析一些讓電腦可以自動「學習」的演算法。機器學習演算法是一類從資料中自動分析獲得規律，並利用規律對未知資料進行預測的演算法。因為學習演算法中涉及了大量的統計學理論，機器學習與推斷統計學聯絡尤為密切，也被稱為統計學習理論。演算法設計方面，機器學習理論關注可以實現的，行之有效的學習演算法。很多推論問題屬於無程式可循難度，所以部分的機器學習研究是開發容易處理的近似演算法。

機器學習已廣泛應用於資料探勘、電腦視覺、自然語言處理、生物特徵辨識、搜尋引擎、醫學診斷、檢測信用卡欺詐、證券市場分析、DNA序列定序、語音和手寫辨識、戰略遊戲和機器人等領域。"""
doc = doc.replace('\n', '')
print(len(doc))
doc

446


'機器學習是人工智慧的一個分支。人工智慧的研究歷史有著一條從以「推理」為重點，到以「知識」為重點，再到以「學習」為重點的自然、清晰的脈絡。顯然，機器學習是實現人工智慧的一個途徑，即以機器學習為手段解決人工智慧中的問題。機器學習在近30多年已發展為一門多領域交叉學科，涉及機率論、統計學、逼近論、凸分析、計算複雜性理論等多門學科。機器學習理論主要是設計和分析一些讓電腦可以自動「學習」的演算法。機器學習演算法是一類從資料中自動分析獲得規律，並利用規律對未知資料進行預測的演算法。因為學習演算法中涉及了大量的統計學理論，機器學習與推斷統計學聯絡尤為密切，也被稱為統計學習理論。演算法設計方面，機器學習理論關注可以實現的，行之有效的學習演算法。很多推論問題屬於無程式可循難度，所以部分的機器學習研究是開發容易處理的近似演算法。機器學習已廣泛應用於資料探勘、電腦視覺、自然語言處理、生物特徵辨識、搜尋引擎、醫學診斷、檢測信用卡欺詐、證券市場分析、DNA序列定序、語音和手寫辨識、戰略遊戲和機器人等領域。'

In [17]:
# 1. 將文章向量化，透過 BERT 語言模型，此處是用[CLS]，但也可以用last_hidden_state每一個word 去做平均取得。

tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')
bert = TFBertModel.from_pretrained('bert-base-chinese')

inputs = tokenizer.encode(doc, return_tensors='tf')
# print(inputs)
doc2vector = bert(inputs)['pooler_output']
print(doc2vector)

Some layers from the model checkpoint at bert-base-chinese were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-chinese.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


tf.Tensor(
[[ 0.02980585 -0.31292656  0.9027152  -0.984225   -0.5525      0.9163109
  -0.15951638  0.97348636 -0.80231243 -0.8116168   0.99078405  0.983601
   0.68423206  0.7631964   0.99701923 -0.9638748   0.94299567  0.9817041
  -0.9812697   0.4736475   0.4045722  -0.9446189  -0.88698804 -0.9858693
   0.96378803  0.81121284  0.9372629   0.7220981  -0.9321696   0.25185204
   0.9287982   0.99244386 -0.02511659 -0.7571809   0.6906712   0.91584873
  -0.9752722   0.36335194 -0.8063998  -0.88280237 -0.08382519  0.8946725
   0.82668275 -0.5608644  -0.5790209  -0.07967257 -0.9988373   0.7886579
   0.39715266  0.3619072   0.7958394  -0.5980953   0.85921854 -0.97696763
   0.09474425 -0.8534039   0.4143035   0.9178072   0.99985445 -0.9452131
   0.99893385  0.92516893 -0.7243568  -0.71710736  0.99425477 -0.3087543
  -0.19610961 -0.2592852   0.61792815  0.9781755  -0.9016027  -0.9202529
   0.9887201   0.67406267 -0.4869594   0.9769172  -0.44899216  0.66852766
  -0.99504614 -0.9906442   0.99782765

In [18]:
# 2. 將文章拆解成句子

pattern = r'[，。]'
sentences = re.split(pattern=pattern, string=doc)[:-1]  # 最後一個是''
sentences

['機器學習是人工智慧的一個分支',
 '人工智慧的研究歷史有著一條從以「推理」為重點',
 '到以「知識」為重點',
 '再到以「學習」為重點的自然、清晰的脈絡',
 '顯然',
 '機器學習是實現人工智慧的一個途徑',
 '即以機器學習為手段解決人工智慧中的問題',
 '機器學習在近30多年已發展為一門多領域交叉學科',
 '涉及機率論、統計學、逼近論、凸分析、計算複雜性理論等多門學科',
 '機器學習理論主要是設計和分析一些讓電腦可以自動「學習」的演算法',
 '機器學習演算法是一類從資料中自動分析獲得規律',
 '並利用規律對未知資料進行預測的演算法',
 '因為學習演算法中涉及了大量的統計學理論',
 '機器學習與推斷統計學聯絡尤為密切',
 '也被稱為統計學習理論',
 '演算法設計方面',
 '機器學習理論關注可以實現的',
 '行之有效的學習演算法',
 '很多推論問題屬於無程式可循難度',
 '所以部分的機器學習研究是開發容易處理的近似演算法',
 '機器學習已廣泛應用於資料探勘、電腦視覺、自然語言處理、生物特徵辨識、搜尋引擎、醫學診斷、檢測信用卡欺詐、證券市場分析、DNA序列定序、語音和手寫辨識、戰略遊戲和機器人等領域']

In [19]:
# 3. 將句子向量化得到sentence2vec

sentence2vec = {}

for s in sentences:
    s_input = tokenizer(s, return_tensors='tf')
    sentence2vec[s] = bert(s_input)['pooler_output']
sentence2vec

{'機器學習是人工智慧的一個分支': <tf.Tensor: shape=(1, 768), dtype=float32, numpy=
 array([[ 0.7645082 ,  0.9952854 ,  0.99095494, -0.9273972 ,  0.25056642,
          0.914284  , -0.5115138 ,  0.9956432 ,  0.8117279 , -0.9369704 ,
          0.9995019 ,  0.99737275,  0.9226132 , -0.04836331,  0.9983111 ,
         -0.9918073 ,  0.89181006,  0.9997184 ,  0.13689053,  0.5023091 ,
          0.9408003 , -0.99996006, -0.9523706 , -0.9886307 ,  0.6316975 ,
          0.8152829 ,  0.9620456 ,  0.7659559 , -0.9976498 ,  0.90442455,
          0.6884256 ,  0.9965725 ,  0.71798676, -0.9991365 , -0.9529468 ,
          0.8955479 , -0.9730849 ,  0.8689999 ,  0.18208854, -0.9821832 ,
         -0.9743564 ,  0.37578288,  0.92802465, -0.9814997 , -0.8720475 ,
         -0.17712605, -0.99993217, -0.9917653 ,  0.17357406,  0.98197204,
         -0.19322039, -0.9990668 ,  0.5008788 , -0.94550645, -0.23805691,
          0.19003808, -0.9902278 ,  0.8013929 ,  0.999982  , -0.9541356 ,
          0.99917907,  0.69827265,  0.00853

In [20]:
# 4. 計算文章與句子之間的相似度(cosine similarity or others)


for s in sentence2vec:
    sentence2vec[s] = cosine_similarity(sentence2vec[s], doc2vector)[0][0]

sentence2vec

{'機器學習是人工智慧的一個分支': 0.7623569,
 '人工智慧的研究歷史有著一條從以「推理」為重點': 0.59828395,
 '到以「知識」為重點': 0.50797117,
 '再到以「學習」為重點的自然、清晰的脈絡': 0.50801206,
 '顯然': 0.37482134,
 '機器學習是實現人工智慧的一個途徑': 0.7307997,
 '即以機器學習為手段解決人工智慧中的問題': 0.70332754,
 '機器學習在近30多年已發展為一門多領域交叉學科': 0.75867635,
 '涉及機率論、統計學、逼近論、凸分析、計算複雜性理論等多門學科': 0.7084698,
 '機器學習理論主要是設計和分析一些讓電腦可以自動「學習」的演算法': 0.6830156,
 '機器學習演算法是一類從資料中自動分析獲得規律': 0.6957466,
 '並利用規律對未知資料進行預測的演算法': 0.57675505,
 '因為學習演算法中涉及了大量的統計學理論': 0.64795136,
 '機器學習與推斷統計學聯絡尤為密切': 0.67719686,
 '也被稱為統計學習理論': 0.6673366,
 '演算法設計方面': 0.6364229,
 '機器學習理論關注可以實現的': 0.631742,
 '行之有效的學習演算法': 0.580939,
 '很多推論問題屬於無程式可循難度': 0.5753635,
 '所以部分的機器學習研究是開發容易處理的近似演算法': 0.6794783,
 '機器學習已廣泛應用於資料探勘、電腦視覺、自然語言處理、生物特徵辨識、搜尋引擎、醫學診斷、檢測信用卡欺詐、證券市場分析、DNA序列定序、語音和手寫辨識、戰略遊戲和機器人等領域': 0.6979066}

In [21]:
# 5. 根據使用需要取得句子(如 top n 個句子 / 或者相似度超過 X % 的所有句子)

# 取得前5個相似度高的句子
top_n = 5
print(sorted(sentence2vec.items(), key=lambda x: x[1], reverse=True)[:top_n])
print()

# 取得threshold > 0.6的句子
print(list(map(lambda x: x if x[1] >= 0.6 else None, sentence2vec.items())))

[('機器學習是人工智慧的一個分支', 0.7623569), ('機器學習在近30多年已發展為一門多領域交叉學科', 0.75867635), ('機器學習是實現人工智慧的一個途徑', 0.7307997), ('涉及機率論、統計學、逼近論、凸分析、計算複雜性理論等多門學科', 0.7084698), ('即以機器學習為手段解決人工智慧中的問題', 0.70332754)]

[('機器學習是人工智慧的一個分支', 0.7623569), None, None, None, None, ('機器學習是實現人工智慧的一個途徑', 0.7307997), ('即以機器學習為手段解決人工智慧中的問題', 0.70332754), ('機器學習在近30多年已發展為一門多領域交叉學科', 0.75867635), ('涉及機率論、統計學、逼近論、凸分析、計算複雜性理論等多門學科', 0.7084698), ('機器學習理論主要是設計和分析一些讓電腦可以自動「學習」的演算法', 0.6830156), ('機器學習演算法是一類從資料中自動分析獲得規律', 0.6957466), None, ('因為學習演算法中涉及了大量的統計學理論', 0.64795136), ('機器學習與推斷統計學聯絡尤為密切', 0.67719686), ('也被稱為統計學習理論', 0.6673366), ('演算法設計方面', 0.6364229), ('機器學習理論關注可以實現的', 0.631742), None, None, ('所以部分的機器學習研究是開發容易處理的近似演算法', 0.6794783), ('機器學習已廣泛應用於資料探勘、電腦視覺、自然語言處理、生物特徵辨識、搜尋引擎、醫學診斷、檢測信用卡欺詐、證券市場分析、DNA序列定序、語音和手寫辨識、戰略遊戲和機器人等領域', 0.6979066)]


In [37]:
# 最後一步就是將其整合起來


def get_doc_summarization(doc, tokenizer, model, top_n=5, threshold=0.5):
    doc = doc.replace('\n', '')
    pattern = r'[，。]'
    sentences = re.split(pattern=pattern, string=doc)[:-1]  # 最後一個是''
    sentence2vec = {}
    
    for s in sentences:
        s_input = tokenizer(s, return_tensors='tf')
        sentence2vec[s] = model(s_input)['pooler_output']
        sentence2vec[s] = cosine_similarity(sentence2vec[s], doc2vector)[0][0]
    
    sorted_list = sorted(sentence2vec.items(), key=lambda x: x[1], reverse=True)[:top_n]
    result = []
    for e in sorted_list:
        if e[1] >= threshold:
            result.append(e)
    return result

tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')
bert = TFBertModel.from_pretrained('bert-base-chinese')
get_doc_summarization(doc, tokenizer, bert)

Some layers from the model checkpoint at bert-base-chinese were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-chinese.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


[('機器學習是人工智慧的一個分支', 0.7623569),
 ('機器學習在近30多年已發展為一門多領域交叉學科', 0.75867635),
 ('機器學習是實現人工智慧的一個途徑', 0.7307997),
 ('涉及機率論、統計學、逼近論、凸分析、計算複雜性理論等多門學科', 0.7084698),
 ('即以機器學習為手段解決人工智慧中的問題', 0.70332754)]

In [38]:
# 儲存模型

bert.save_weights('bert-summarization')
#sequential_model.save_weights("ckpt")
#load_status = sequential_model.load_weights("ckpt")

In [39]:
bert.load_weights('bert-summarization')

In [40]:
get_doc_summarization(doc, tokenizer, bert)

[('機器學習是人工智慧的一個分支', 0.7623569),
 ('機器學習在近30多年已發展為一門多領域交叉學科', 0.75867635),
 ('機器學習是實現人工智慧的一個途徑', 0.7307997),
 ('涉及機率論、統計學、逼近論、凸分析、計算複雜性理論等多門學科', 0.7084698),
 ('即以機器學習為手段解決人工智慧中的問題', 0.70332754)]